## 2.0 Time series clustering

In this notebook we try to cluster the hole dataset, using the model trained in the previous one

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from tensorflow.keras import layers, Model
import tensorflow as tf
from tslearn.clustering import TimeSeriesKMeans
from tslearn.barycenters import dtw_barycenter_averaging as dtw_avg
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline   

In [2]:
df = np.empty((len(os.listdir('../test_data/new_data/')),385,4))
pixels = []
i = 0



for file in os.listdir('../test_data/new_data/'):
    df[i,:,:] = pd.read_csv(os.path.join('../test_data/new_data/', file), header=None)
    i += 1
    pixels.append(file)
    
df = df[:,1:,:]

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [4]:
encoder = tf.keras.models.load_model('./my_encoder')

data = encoder.predict(df)
data = data.reshape((len(os.listdir('../test_data/new_data/')), 384))

data = pd.DataFrame(data, index = pixels)

409/409 [==============================] - 1s 3ms/step


In [5]:
model = TimeSeriesKMeans(n_clusters=7, metric="dtw", max_iter=100)
model.fit(data)

In [ ]:
results = pd.DataFrame()
results['group'] = model.predict(data)

coord_x = []
coord_y = []

for i in range(len(pixels)):
    pixels[i] = pixels[i].replace('.csv', '')
    coord_x.append(float(pixels[i].split(',')[1]))
    coord_y.append(float(pixels[i].split(',')[0]))

results['coord_x'] = coord_x
results['coord_y'] = coord_y

In [ ]:
# From GeoPandas, our world map data
worldmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Creating axes and plotting world map
fig, ax = plt.subplots(figsize=(12, 6))
worldmap.plot(color="lightgrey", ax=ax)

for group, color in zip(results['group'].unique(), ['blue', 'red', 'green', 'orange', 'purple', 'yellow', 'brown']):
    plt.scatter(x = results.coord_x[results['group'] == group], y = results.coord_y[results['group'] == group], color=color, label=f"Group {group}")

# Creating axis limits and title
plt.xlim([-180, 180])
plt.ylim([-90, 90])

plt.title('Pixels classification on climate type')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.show()